In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset

In [4]:
df = pd.read_csv("../data/interim/hr_analytics_merged.csv")

#### Handle missing values

In [8]:
missing_cols = df.columns[df.isnull().any()].to_list()
print(f"Kolom dengan Missing Values: {missing_cols}")
print(df[missing_cols].isnull().sum())

Kolom dengan Missing Values: ['NumCompaniesWorked', 'TotalWorkingYears', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']
NumCompaniesWorked         19
TotalWorkingYears           9
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64


Data numerik (NumCompaniesWorked dan TotalWorkingYears) diisi dengan nilai median tiap kolom untuk menghindari nilai outlier

In [ ]:
df['NumCompaniesWorked'] = df['NumCompaniesWorked'].fillna(df['NumCompaniesWorked'].median())
df['TotalWorkingYears'] = df['TotalWorkingYears'].fillna(df['TotalWorkingYears'].median())